In [ ]:
####no layer_again

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from io import BytesIO
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input
from keras.preprocessing.image import ImageDataGenerator
from keras import Model, layers, optimizers, callbacks, applications
from keras.optimizers import Adam, SGD
from keras import Sequential
import pandas as pd
import csv
import keras
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from PIL import ImageFile
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard


from tensorflow.python.client import device_lib 
print('check device:', device_lib.list_local_devices())
ImageFile.LOAD_TRUNCATED_IMAGES = True
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

print('creating directories')
curr_dir = "/content/drive/My Drive"
train_dir = os.path.join(curr_dir,"the whole memotion dir/train/Overall_sentiment")
validation_dir = os.path.join(curr_dir,"the whole memotion dir/eval/Overall_sentiment")
print('train_dir', train_dir)
print('validation_dir', validation_dir)

#CHANGE
####other constants
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)
NUM_CLASSES = 5
FILENAME = 'BiT-M_result_1'
EPOCHS = 300
saved_area = os.path.join(curr_dir, "saved_models_2/BiT-M/result_1")
CHKPT_FILEPATH = os.path.join(saved_area,'cp-{epoch:03d}-{val_accuracy:.5f}.h5')

MODULE_HANDLE ="https://tfhub.dev/google/bit/m-r50x1/ilsvrc2012_classification/1"
CONV_BASE = hub.KerasLayer(MODULE_HANDLE, trainable=True)
OPTIMIZER = keras.optimizers.SGD(learning_rate=0.003, momentum=0.9)
A_PATIENCE = 20
B_PATIENCE = 10

check device: [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17045959484168803942
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17498158776333111151
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7378975716800342276
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15701463552
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8088038754459734858
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.

In [ ]:
do_fine_tuning = False

print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=TARGET_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dense(NUM_CLASSES, activation ="softmax")
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.003, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])


creating the models
Using https://tfhub.dev/google/bit/m-r50x1/ilsvrc2012_classification/1 with input size (224, 224)
Building model with https://tfhub.dev/google/bit/m-r50x1/ilsvrc2012_classification/1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1000)              25549352  
_________________________________________________________________
dense (Dense)                (None, 128)               128128    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 25,678,125
Trainable params: 128,773
Non-trainable params: 25,549,352
_________________________________________________________________


In [ ]:
print('augmentation')  
train_datagen = ImageDataGenerator()
####augmentation
test_datagen = ImageDataGenerator()

print('train generator')
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

print('validation generator')
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

#CHANGE
print('checkpointing')
saved_area ='/content/drive/My Drive/saved_models/BiTResnet_confirmwithlayer_fromscratch_300epochs'
my_callbacks = [
tf.keras.callbacks.EarlyStopping(patience=10), 
tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(saved_area, 'S8.1.1_overall_sentiment_run1_BiT_resnet_model.{epoch:02d}-{val_loss:.2f}.h5'), monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto'), 
tf.keras.callbacks.TensorBoard(log_dir='./logs')]

#CHANGE EPOCH
print('history')
history = model.fit(
    train_generator, 
    callbacks = my_callbacks,
    epochs=300,
    validation_data=validation_generator,
    verbose=2)



augmentation
train generator
Found 5995 images belonging to 5 classes.
validation generator
Found 914 images belonging to 5 classes.
checkpointing
history
Epoch 1/300


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "



Epoch 00001: val_loss improved from inf to 1.47924, saving model to /content/drive/My Drive/saved_models/BiTResnet_confirmwithlayer_fromscratch_300epochs/S8.1.1_overall_sentiment_run1_BiT_resnet_model.01-1.48.h5
188/188 - 2346s - loss: 1.4880 - accuracy: 0.4395 - val_loss: 1.4792 - val_accuracy: 0.4508
Epoch 2/300

Epoch 00002: val_loss improved from 1.47924 to 1.47915, saving model to /content/drive/My Drive/saved_models/BiTResnet_confirmwithlayer_fromscratch_300epochs/S8.1.1_overall_sentiment_run1_BiT_resnet_model.02-1.48.h5
188/188 - 130s - loss: 1.4826 - accuracy: 0.4467 - val_loss: 1.4791 - val_accuracy: 0.4508
Epoch 3/300

Epoch 00003: val_loss improved from 1.47915 to 1.47907, saving model to /content/drive/My Drive/saved_models/BiTResnet_confirmwithlayer_fromscratch_300epochs/S8.1.1_overall_sentiment_run1_BiT_resnet_model.03-1.48.h5
188/188 - 131s - loss: 1.4822 - accuracy: 0.4472 - val_loss: 1.4791 - val_accuracy: 0.4508
Epoch 4/300

Epoch 00004: val_loss improved from 1.4790

In [ ]:
####checkpoints, and show validation accuracy
print('evaluating')

print('val_preds')
predictions = model.predict(validation_generator)
val_preds = np.argmax(predictions, axis=-1)
df1 = pd.DataFrame(val_preds)
csv_filename = 'val_preds.csv'
df1.to_csv(os.path.join(saved_area, csv_filename), index= True)
print(val_preds)

print('val_trues')
val_trues = validation_generator.classes
df2 = pd.DataFrame(val_trues)
csv_filename = 'val_trues.csv'
df2.to_csv(os.path.join(saved_area, csv_filename), index= True)
print(val_trues)

print('confusion matrix')
cm = confusion_matrix(val_trues, val_preds)
df3 = pd.DataFrame(cm)
csv_filename = 'confusion_matrix.csv'
df3.to_csv(os.path.join(saved_area, csv_filename), index= True)
print(cm)

print('saving to csv')
report = pd.DataFrame(classification_report(val_trues, val_preds, output_dict=True)).transpose()
print(report)
#CHANGE
csv_filename = 'S8_memotion_overall_sentiment_run1_BiTlayercfm_.csv'
report.to_csv(os.path.join(saved_area, csv_filename), index= True)
    
history = history.history
to_be_saved = 'Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1])
print(to_be_saved)
            
print('append to csv file')
with open(os.path.join(saved_area, csv_filename),'a') as fd:
    fd.write(to_be_saved)
    fd.close()
    
###save model
print('saving model')
acc=history['val_acc'][-1]
#CHANGE
model.save(os.path.join(saved_area, 'S8_memotion_overall_sentiment_run1_cfmBiTlayers_{}.h5'.format(acc)))

evaluating
val_preds
[2 2 2 1 4 0 2 2 1 2 2 2 4 1 2 1 2 2 2 2 2 0 2 2 4 2 4 1 2 2 1 4 2 1 1 1 2
 2 2 4 1 2 1 2 2 2 4 2 2 1 2 1 2 0 2 2 1 2 1 2 2 4 1 2 4 1 2 1 2 2 1 2 4 1
 2 1 2 1 1 2 1 2 2 4 1 2 4 1 4 2 1 4 1 2 1 1 4 2 1 2 4 2 1 2 1 2 2 2 2 2 2
 2 2 2 1 4 2 1 2 2 1 2 2 1 4 2 2 1 2 2 1 4 2 1 2 2 4 2 2 2 1 2 2 2 0 2 2 2
 2 1 2 1 1 1 2 1 2 4 1 2 4 2 1 2 4 1 2 2 4 4 1 2 1 1 2 2 2 0 2 0 1 1 1 2 2
 1 2 2 2 1 2 2 2 1 1 1 2 1 2 2 4 2 0 2 0 2 2 1 1 0 2 4 2 2 1 4 2 2 2 2 2 4
 2 2 2 2 0 4 1 1 1 2 2 2 1 1 2 0 2 2 4 1 2 1 2 1 1 0 2 2 4 2 4 2 2 2 4 2 2
 2 4 2 2 2 2 2 2 4 4 1 2 1 2 2 1 1 4 1 2 2 2 0 1 2 2 2 1 2 0 4 2 1 2 2 2 1
 2 1 2 1 2 2 1 2 2 2 1 2 2 0 1 4 2 1 1 2 2 1 1 1 2 1 2 2 2 2 4 2 1 1 2 1 2
 4 1 2 2 2 2 2 2 2 1 4 4 1 4 2 4 4 1 1 1 2 1 2 1 2 2 2 2 2 1 4 4 2 1 4 2 2
 2 2 2 1 2 0 4 1 2 1 0 2 4 1 1 4 1 2 1 2 1 1 4 2 2 4 2 2 2 2 2 2 4 2 2 1 1
 1 2 2 2 1 4 2 1 1 2 1 2 2 2 2 1 2 4 4 4 1 2 1 2 2 2 1 1 2 4 2 2 1 2 2 2 2
 2 2 2 1 2 1 2 1 2 2 1 1 2 1 2 4 2 2 2 4 2 4 4 4 2 2 4 2 2 2 2 2 1 2 1 2 1
 2 2

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')